In [4]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [5]:
spark

In [6]:
sc.stop()

In [7]:
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession,HiveContext

### Creating New Spark Instance

In [8]:
config = SparkConf().setAppName('SparkHiveSession').setMaster('local[4]')
sc = SparkContext.getOrCreate(conf=config)

In [9]:
sc

<SparkContext master=local[4] appName=SparkHiveSession>

### a) Creating Connection to Spark and Hive

In [10]:
spark = (SparkSession.builder.appName("pysparkhive-integration")
         .config('spark.sql.warehouse.dir', '/user/hive/warehouse/')
         .enableHiveSupport().getOrCreate())

In [11]:
spark

### b) Creating Dataframe using csv file

In [12]:
flight_df = spark.read.csv("file:///home/hadoop/Downloads/Flights_Delay.csv", inferSchema = True, header = True)

In [11]:
flight_df.show()

+---+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
| ID|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+---+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+-----------

In [12]:
flight_df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true

### C) Creating Hive table

In [18]:
spark.sql("""
CREATE TABLE if not exists flights_table (ID INT,YEAR INT,MONTH INT,DAY INT,DAY_OF_WEEK INT,AIRLINE STRING,
FLIGHT_NUMBER STRING,TAIL_NUMBER STRING,ORIGIN_AIRPORT STRING,DESTINATION_AIRPORT STRING,SCHEDULED_DEPARTURE INT,
DEPARTURE_TIME INT,DEPARTURE_DELAY INT,TAXI_OUT INT,WHEELS_OFF INT,SCHEDULED_TIME INT,ELAPSED_TIME INT,AIR_TIME INT,
DISTANCE INT,WHEELS_ON INT,TAXI_IN INT,SCHEDULED_ARRIVAL INT,ARRIVAL_TIME INT,ARRIVAL_DELAY INT,DIVERTED INT,
CANCELLED INT,CANCELLATION_REASON STRING,AIR_SYSTEM_DELAY INT,SECURITY_DELAY INT,AIRLINE_DELAY INT,LATE_AIRCRAFT_DELAY INT,WEATHER_DELAY INT)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
tblProperties("skip.header.line.count" = 1)""")

DataFrame[]

### d) Describe Table Schema & show top 10 dataset

In [26]:
flight_df.createOrReplaceTempView('flight_temp_table')

In [27]:
spark.sql("""
INSERT INTO flights_table
SELECT * FROM flight_temp_table""")

DataFrame[]

In [30]:
spark.sql("describe flights_table").show()

+-------------------+---------+-------+
|           col_name|data_type|comment|
+-------------------+---------+-------+
|                 ID|      int|   null|
|               YEAR|      int|   null|
|              MONTH|      int|   null|
|                DAY|      int|   null|
|        DAY_OF_WEEK|      int|   null|
|            AIRLINE|   string|   null|
|      FLIGHT_NUMBER|   string|   null|
|        TAIL_NUMBER|   string|   null|
|     ORIGIN_AIRPORT|   string|   null|
|DESTINATION_AIRPORT|   string|   null|
|SCHEDULED_DEPARTURE|      int|   null|
|     DEPARTURE_TIME|      int|   null|
|    DEPARTURE_DELAY|      int|   null|
|           TAXI_OUT|      int|   null|
|         WHEELS_OFF|      int|   null|
|     SCHEDULED_TIME|      int|   null|
|       ELAPSED_TIME|      int|   null|
|           AIR_TIME|      int|   null|
|           DISTANCE|      int|   null|
|          WHEELS_ON|      int|   null|
+-------------------+---------+-------+
only showing top 20 rows



In [31]:
spark.sql("select * from flights_table limit 10").show()

+---+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|ID |YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+---+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+-----------

### e) Applying Query performance optimization techniques

In [36]:
flight_df.write.partitionBy('YEAR','MONTH').parquet("file:///home/hadoop/Downloads/Flight_Data")

In [37]:
parquet_data = spark.read.parquet("file:///home/hadoop/Downloads/Flight_Data")

In [38]:
flight_df.cache()

DataFrame[ID: int, YEAR: int, MONTH: int, DAY: int, DAY_OF_WEEK: int, AIRLINE: string, FLIGHT_NUMBER: int, TAIL_NUMBER: string, ORIGIN_AIRPORT: string, DESTINATION_AIRPORT: string, SCHEDULED_DEPARTURE: int, DEPARTURE_TIME: int, DEPARTURE_DELAY: int, TAXI_OUT: int, WHEELS_OFF: int, SCHEDULED_TIME: int, ELAPSED_TIME: int, AIR_TIME: int, DISTANCE: int, WHEELS_ON: int, TAXI_IN: int, SCHEDULED_ARRIVAL: int, ARRIVAL_TIME: int, ARRIVAL_DELAY: int, DIVERTED: int, CANCELLED: int, CANCELLATION_REASON: string, AIR_SYSTEM_DELAY: int, SECURITY_DELAY: int, AIRLINE_DELAY: int, LATE_AIRCRAFT_DELAY: int, WEATHER_DELAY: int]

### f) average analysis delay caused by airlines

In [43]:
spark.sql("Select AIRLINE, avg(ARRIVAL_DELAY) as Average_Delay from flights_table group by AIRLINE order by Average_Delay").show()

+-------+------------------+
|AIRLINE|     Average_Delay|
+-------+------------------+
|     AS|-1.531766200762389|
|     DL|2.8144726712856043|
|     WN| 3.697840458351697|
|     HA| 4.072423398328691|
|     VX| 5.128571428571429|
|     US| 5.977315185481719|
|     UA| 6.697221614526362|
|     AA| 8.386631979187513|
|     OO|10.154792043399638|
|     EV|10.884270870655678|
|     B6| 13.95852534562212|
|     NK|14.206426484907498|
|     MQ|19.231592604605904|
|     F9|24.103448275862068|
+-------+------------------+



### g) Days of months with respected to average of arrival delay

In [44]:
spark.sql("select DAY, avg(ARRIVAL_DELAY) as Average_Delay from flights_table group by DAY order by Average_Delay").show()

+---+--------------------+
|DAY|       Average_Delay|
+---+--------------------+
| 31|  -1.196594427244582|
| 10|-0.04705882352941...|
| 29| 0.07971014492753623|
| 14|  1.3299319727891157|
| 19|  1.6344282238442822|
|  7|  2.8309417040358746|
| 15|   2.966753585397653|
| 28|   3.257425742574257|
| 13|  3.3769751693002257|
| 18|  3.5693430656934306|
| 20|  3.8770149253731345|
| 11|  3.9912935323383083|
| 23|   4.207086133170434|
|  9|   4.421887390959556|
| 30|   4.471478463329452|
| 27|   4.706711409395973|
| 25|   4.903708523096942|
|  8|   5.232349165596919|
| 24|   5.737543859649123|
| 22|   6.550920245398773|
+---+--------------------+
only showing top 20 rows



### h) Arrange weekday with respect to the average arrival delay

In [46]:
spark.sql("select DAY_OF_WEEK, avg(ARRIVAL_DELAY) as AVG_DELAY from flights_table group by DAY_OF_WEEK order by AVG_DELAY").show()

+-----------+------------------+
|DAY_OF_WEEK|         AVG_DELAY|
+-----------+------------------+
|          6| 4.888689138576779|
|          3| 5.587079407806191|
|          5| 6.010538373424971|
|          4| 7.174969021065675|
|          2| 8.033644102148358|
|          7|10.110840438489646|
|          1|10.807447207297264|
+-----------+------------------+



### i) Arrange Day of month as per cancellations done in descending

In [51]:
spark.sql("select DAY,MONTH from flights_table where CANCELLED = 1 group by DAY,MONTH order by DAY desc").show()

+---+-----+
|DAY|MONTH|
+---+-----+
| 31|    1|
| 30|    1|
| 29|    1|
| 28|    2|
| 28|    1|
| 27|    2|
| 27|    1|
| 26|    2|
| 26|    1|
| 25|    2|
| 25|    1|
| 24|    2|
| 24|    1|
| 23|    2|
| 23|    1|
| 22|    1|
| 22|    2|
| 21|    1|
| 21|    2|
| 20|    2|
+---+-----+
only showing top 20 rows



### j) Top 10 busiest airport with respect to day of week

In [1]:
spark.sql("""
WITH flight_counts AS (
SELECT ORIGIN_AIRPORT AS AIRPORT,DAY_OF_WEEK, COUNT(*) AS
NUM_FLIGHTS
FROM flights_table
GROUP BY ORIGIN_AIRPORT,DAY_OF_WEEK
UNION ALL
SELECT DESTINATION_AIRPORT AS AIRPORT,DAY_OF_WEEK,COUNT(*)
AS NUM_FLIGHTS
FROM flights_table
GROUP BY DESTINATION_AIRPORT,DAY_OF_WEEK
),
airport_totals AS (
SELECT
AIRPORT,
DAY_OF_WEEK,
SUM(NUM_FLIGHTS) AS TOTAL_FLIGHTS
FROM flight_counts
GROUP BY AIRPORT,DAY_OF_WEEK
),
ranked_airports AS (
SELECT
AIRPORT,
DAY_OF_WEEK,
TOTAL_FLIGHTS,
ROW_NUMBER() OVER (PARTITION BY DAY_OF_WEEK ORDER BY
TOTAL_FLIGHTS DESC) AS RANK
FROM airport_totals
)
SELECT
AIRPORT,
TOTAL_FLIGHTS,
RANK
FROM ranked_airports
WHERE RANK <= 10
ORDER BY DAY_OF_WEEK,RANK
LIMIT 10
""").show()

+-------+-------------+----+
|AIRPORT|TOTAL_FLIGHTS|RANK|
+-------+-------------+----+
|    ATL|         1106|   1|
|    ORD|          844|   2|
|    DFW|          818|   3|
|    LAX|          631|   4|
|    DEN|          613|   5|
|    IAH|          494|   6|
|    PHX|          485|   7|
|    SFO|          466|   8|
|    LAS|          398|   9|
|    MSP|          382|  10|
+-------+-------------+----+



### k) Find airlines that make maximum number of cancellation

In [11]:
spark.sql("select AIRLINE, count(*) as MAX_CANCELLATION from flights_table where CANCELLED = 1 group by AIRLINE order by MAX_CANCELLATION desc LIMIT 10").show()

+-------+----------------+
|AIRLINE|MAX_CANCELLATION|
+-------+----------------+
|     MQ|             414|
|     WN|             358|
|     EV|             312|
|     AA|             241|
|     DL|             177|
|     US|             169|
|     OO|             153|
|     B6|             145|
|     UA|             122|
|     NK|              21|
+-------+----------------+



### l) find and order airlines in decending that make most number of diversion

In [5]:
spark.sql("select AIRLINE, count(*) as MAX_DIVERTED from flights_table where DIVERTED = 1 group by AIRLINE order by MAX_DIVERTED desc").show()

+-------+------------+
|AIRLINE|MAX_DIVERTED|
+-------+------------+
|     WN|          35|
|     OO|          25|
|     EV|          22|
|     DL|          18|
|     B6|          16|
|     AA|          12|
|     US|           9|
|     UA|           8|
|     MQ|           5|
|     HA|           1|
+-------+------------+



### m) Find day of month that see most number of diversion

In [40]:
spark.sql("select DAY,MONTH, count(*) as MAX_DIVERTED from flights_table where DIVERTED = 1 group by DAY,MONTH order by MAX_DIVERTED desc").show()

+---+-----+------------+
|DAY|MONTH|MAX_DIVERTED|
+---+-----+------------+
|  2|    2|           9|
| 14|    2|           7|
|  5|    3|           7|
|  1|    3|           7|
|  4|    3|           6|
|  2|    3|           6|
| 18|    1|           5|
|  7|    1|           5|
|  1|    2|           5|
| 11|    1|           5|
| 23|    2|           5|
| 30|    1|           5|
|  8|    1|           4|
|  9|    2|           4|
| 21|    2|           4|
| 31|    1|           3|
| 12|    1|           3|
| 16|    2|           3|
|  9|    1|           3|
|  3|    3|           3|
+---+-----+------------+
only showing top 20 rows



### n)calculate mean and standard deviation of departure delay

In [1]:
from pyspark.sql.functions import (col,floor)

In [13]:
arrival_delay_min = flight_df.withColumn('DEPARTURE_DELAY_MINUTES', col("DEPARTURE_DELAY")%60).withColumn("ARRIVAL_DELAY_MINUTES",col("ARRIVAL_DELAY")%60)

In [14]:
arrival_delay_min.createOrReplaceTempView("flight_df_delay_in_minutes")

In [18]:
spark.sql("select avg(DEPARTURE_DELAY_MINUTES) as MEAN, std(DEPARTURE_DELAY_MINUTES) as STD_DELAY from flight_df_delay_in_minutes").show()

+----------------+------------------+
|            MEAN|         STD_DELAY|
+----------------+------------------+
|4.76619112108788|15.059861202574895|
+----------------+------------------+



### o) calculate mean and standard deviation of arrival delay

In [20]:
spark.sql("select avg(ARRIVAL_DELAY_MINUTES) as MEAN, std(ARRIVAL_DELAY_MINUTES) as STANDARD_DEVIATION from flight_df_delay_in_minutes").show()

+------------------+------------------+
|              MEAN|STANDARD_DEVIATION|
+------------------+------------------+
|0.7675518641290179|19.194346550499723|
+------------------+------------------+



### p) finding all diverted route from source to destination airport & which route is the most diverted

In [32]:
spark.sql("""
with DIVERT_ROUTE AS(
	select ORIGIN_AIRPORT,DESTINATION_AIRPORT,
	count(*) AS DIVERT_COUNT
	from flights_table
	where DIVERTED = 1
	group by ORIGIN_AIRPORT,DESTINATION_AIRPORT
),
MAX_DIVERTED_ROUTE AS(
	select ORIGIN_AIRPORT,DESTINATION_AIRPORT,DIVERT_COUNT
	from DIVERT_ROUTE
	order by DIVERT_COUNT desc
	LIMIT 1
)
select dir.ORIGIN_AIRPORT,dir.DESTINATION_AIRPORT,dir.DIVERT_COUNT,
case WHEN dir.ORIGIN_AIRPORT = mdir.ORIGIN_AIRPORT and dir.DESTINATION_AIRPORT = mdir.DESTINATION_AIRPORT
then 'Most Diverted Route' else '' end AS RouteType
from DIVERT_ROUTE dir
left join MAX_DIVERTED_ROUTE mdir
on dir.ORIGIN_AIRPORT = mdir.ORIGIN_AIRPORT and dir.DESTINATION_AIRPORT = mdir.DESTINATION_AIRPORT
order by dir.DIVERT_COUNT desc
""").show()

+--------------+-------------------+------------+-------------------+
|ORIGIN_AIRPORT|DESTINATION_AIRPORT|DIVERT_COUNT|          RouteType|
+--------------+-------------------+------------+-------------------+
|           STT|                PHL|           2|                   |
|           PHL|                SAN|           2|                   |
|           HOU|                DAL|           2|                   |
|           TPA|                LGA|           2|Most Diverted Route|
|           IAH|                ASE|           2|                   |
|           JFK|                EGE|           2|                   |
|           JFK|                SEA|           2|                   |
|           ORD|                ASE|           2|                   |
|           CLT|                IAH|           2|                   |
|           FLL|                BWI|           1|                   |
|           SLC|                RDM|           1|                   |
|           SFO|    

### q)Finding AIRLINES with its total flight count, total number of flights arrival delayed by more than 30 Minutes, % of such flights delayed by more than 30 minutes when it is not Weekends with minimum count of flights from Airlines by more than 10. Also Exclude some of Airlines 'AK', 'HI', 'PR', 'VI' and arrange output in descending order by % of such count of flights. 

In [33]:
spark.sql("""
WITH FILTER_FLIGHT AS (
	select
    	AIRLINE,
    	count(*) AS TOTAL_FLIGHTS,
    	sum(case when ARRIVAL_DELAY > 30 and DAY_OF_WEEK NOT IN (6, 7) then 1 else 0 end) as FLIGHT_ARRIVAL_DELAY
	from flights_table
	where AIRLINE not in ('AK', 'HI', 'PR', 'VI')
	group by AIRLINE
	having TOTAL_FLIGHTS > 10
),
DELAY_PERCENT AS (
	select
    	AIRLINE,
    	TOTAL_FLIGHTS,
    	FLIGHT_ARRIVAL_DELAY,
    	(FLIGHT_ARRIVAL_DELAY / TOTAL_FLIGHTS) * 100 AS PERCENT_DELAY
	from FILTER_FLIGHT
)
select
	AIRLINE,
	TOTAL_FLIGHTS,
	FLIGHT_ARRIVAL_DELAY,
	PERCENT_DELAY
from DELAY_PERCENT
order by PERCENT_DELAY desc
""").show()

+-------+-------------+--------------------+------------------+
|AIRLINE|TOTAL_FLIGHTS|FLIGHT_ARRIVAL_DELAY|     PERCENT_DELAY|
+-------+-------------+--------------------+------------------+
|     F9|          794|                 139|17.506297229219143|
|     MQ|         3502|                 601| 17.16162193032553|
|     B6|         2548|                 360|14.128728414442701|
|     NK|         1048|                 139|13.263358778625955|
|     EV|         5916|                 665|11.240703177822853|
|     OO|         5708|                 633|11.089698668535389|
|     UA|         4701|                 497| 10.57221867687726|
|     AA|         5250|                 484| 9.219047619047618|
|     VX|          573|                  47| 8.202443280977311|
|     US|         3925|                 310| 7.898089171974522|
|     DL|         7989|                 592| 7.410189009888597|
|     WN|        11738|                 869|  7.40330550349293|
|     AS|         1586|                 

### r) Finding AIRLINES with its total flight count with total number of flights departure delayed by less than 30 Minutes, % of such flights delayed by less than 30 minutes when it is Weekends with minimum count of flights from Airlines by more than 10. Also Exclude some of Airlines 'AK', 'HI', 'PR', 'VI' and arrange output in descending order by % of such count of flights. 

In [35]:
spark.sql("""
WITH FILTER_FLIGHT AS (
	select
    	AIRLINE,
    	count(*) AS TOTAL_FLIGHTS,
    	sum(case when ARRIVAL_DELAY < 30 and DAY_OF_WEEK in (6, 7) then 1 else 0 end) as FLIGHT_ARRIVAL_DELAY
	from flights_table
	where AIRLINE not in ('AK', 'HI', 'PR', 'VI')
	group by AIRLINE
	having TOTAL_FLIGHTS > 10
),
DELAY_PERCENT AS (
	select
    	AIRLINE,
    	TOTAL_FLIGHTS,
    	FLIGHT_ARRIVAL_DELAY,
    	(FLIGHT_ARRIVAL_DELAY / TOTAL_FLIGHTS) * 100 AS PERCENT_DELAY
	from FILTER_FLIGHT
)
select
	AIRLINE,
	TOTAL_FLIGHTS,
	FLIGHT_ARRIVAL_DELAY,
	PERCENT_DELAY
from DELAY_PERCENT
order by PERCENT_DELAY desc
""").show()

+-------+-------------+--------------------+------------------+
|AIRLINE|TOTAL_FLIGHTS|FLIGHT_ARRIVAL_DELAY|     PERCENT_DELAY|
+-------+-------------+--------------------+------------------+
|     AS|         1586|                 411|25.914249684741485|
|     HA|          722|                 176| 24.37673130193906|
|     NK|         1048|                 253|24.141221374045802|
|     DL|         7989|                1825|22.843910376768058|
|     AA|         5250|                1194|22.742857142857144|
|     WN|        11738|                2654|22.610325438745953|
|     VX|          573|                 128|22.338568935427574|
|     US|         3925|                 847|21.579617834394902|
|     OO|         5708|                1211| 21.21583742116328|
|     B6|         2548|                 536|21.036106750392463|
|     UA|         4701|                 950| 20.20846628376941|
|     EV|         5916|                1173| 19.82758620689655|
|     MQ|         3502|                 

### s) When is the best time of the day/day of week/time of a year to fly with minimum delay

In [36]:
spark.sql("select DAY_OF_WEEK,avg(ARRIVAL_DELAY) as AVG from flights_table group by DAY_OF_WEEK order by AVG asc Limit 1").show()

+-----------+-----------------+
|DAY_OF_WEEK|              AVG|
+-----------+-----------------+
|          6|4.888689138576779|
+-----------+-----------------+



### t)Which airlines are best airline to travel considering number of cancellations, arrival, departure delays and all reasons affecting performance of airline industry

In [38]:
spark.sql("""
WITH FLIGHT_DATA AS(
SELECT
	AIRLINE,COUNT(*) AS TOTAL_FLIGHT,
	SUM(CASE WHEN CANCELLED=1 THEN 1 ELSE 0 END) AS TOTAL_CANCELLED,
	SUM(CASE WHEN ARRIVAL_DELAY IS NOT NULL THEN ARRIVAL_DELAY ELSE 0 END) AS AVG_ARRIVAL_DELAY,
	SUM(CASE WHEN DEPARTURE_DELAY IS NOT NULL THEN DEPARTURE_DELAY ELSE 0 END) AS AVG_DEPARTURE_DELAY
FROM flights_table
GROUP BY AIRLINE
),
RANKED_AIRLINE AS(
SELECT
	AIRLINE,
	TOTAL_FLIGHT,
	TOTAL_CANCELLED,
	AVG_ARRIVAL_DELAY,
	AVG_DEPARTURE_DELAY,
	RANK() OVER (ORDER BY TOTAL_CANCELLED ASC,AVG_ARRIVAL_DELAY ASC,AVG_DEPARTURE_DELAY ASC) AS RANK
FROM
	FLIGHT_DATA
)

SELECT
	AIRLINE,
	TOTAL_FLIGHT,
	TOTAL_CANCELLED,
	AVG_ARRIVAL_DELAY,
	AVG_DEPARTURE_DELAY
FROM
	RANKED_AIRLINE
order by
	RANK asc
limit 10
""").show()

+-------+------------+---------------+-----------------+-------------------+
|AIRLINE|TOTAL_FLIGHT|TOTAL_CANCELLED|AVG_ARRIVAL_DELAY|AVG_DEPARTURE_DELAY|
+-------+------------+---------------+-----------------+-------------------+
|     HA|         722|              3|             2924|                851|
|     F9|         794|             11|            18873|              18412|
|     AS|        1586|             12|            -2411|               3680|
|     VX|         573|             13|             2872|               5520|
|     NK|        1048|             21|            14590|              16017|
|     UA|        4701|            122|            30613|              65534|
|     B6|        2548|            145|            33319|              38613|
|     OO|        5708|            153|            56156|              64516|
|     US|        3925|            169|            22397|              29375|
|     DL|        7989|            177|            21936|              77642|